In [1]:
!pip install qdrant-client sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.2/377.2 kB 7.3 MB/s eta 0:00:00


# Initialize Client

In [3]:
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient, models

from google.colab import userdata
client = QdrantClient(url=userdata.get("CLUSTER_ENDPOINT"), api_key=userdata.get("QDRANT_API"))

encoder = SentenceTransformer("all-MiniLM-L6-v2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Step 2. Prepare your dataset

In [4]:
# Dataset de Noticias para Qdrant (Día 1)
my_dataset = [
    {
        "title": "Liberia: Senator Crayton Duncan Indicts Legislature for National Poverty",
        "description": """Senator Crayton O. Duncan of Sinoe County has sparked a national debate by
        attributing 80% of Liberia's underdevelopment to the Legislative body's failure to oversee
        the Executive. While public perception blames corruption for the country's state, Duncan
        argues that the lack of accountability and the 'appeasement' of the President by elected
        officials are the primary drivers of resource exploitation and poverty. He specifically
        criticized both former President Weah and current President Boakai for maintaining
        the 'old order' of personal enrichment over public service.""",
        "category": "Politics",
        "source": "frontpageafricaonline",
        "pub_date": "2026-01-29"
    },
    {
        "title": "Thailand Budget Bureau Warns of Debt Ceiling Risks Amid Campaign Pledges",
        "description": """Thailand's Budget Bureau has flagged concerns over political parties' campaign
        promises for the 2026 election, which total an estimated 25 trillion baht. The bureau warns
        that proposed universal welfare schemes, electricity subsidies, and infrastructure projects
        could push the public-debt-to-GDP ratio beyond the 70% legal limit by 2027. Major parties
        like Klatham, the Democrats, and the People's Party have proposed massive spending plans
        that may conflict with long-term fiscal discipline and sovereign credit ratings.""",
        "category": "Economy",
        "source": "nationthailand",
        "pub_date": "2026-01-29"
    },
    {
        "title": "Review of 'Reforming Lessons': The Impact of Nick Gibb on British Education",
        "description": """A look into the legacy of former British schools minister Nick Gibb, as detailed
        in the new book 'Reforming Lessons'. The text explores how Gibb and Michael Gove implemented
        evidence-led reforms, including phonics-based reading and the 'warm-strict' discipline
        model, which transformed English schools after 2010. The book credits the success of the
        Free Schools movement and the move away from 'child-centered' learning towards a knowledge-based
        curriculum, despite opposition from educational orthodoxies.""",
        "category": "Education",
        "source": "spectator_au",
        "pub_date": "2026-01-29"
    },
    {
        "title": "True Crime Drama 'Quiz' Hits Netflix: The Story of the Coughing Major",
        "description": """The critically acclaimed ITV drama 'Quiz' is arriving on Netflix, dramatizing
        the 2001 'Who Wants to Be a Millionaire?' cheating scandal. Starring Michael Sheen as Chris
        Tarrant and Matthew Macfadyen as Charles Ingram, the series follows the 'Coughing Major'
        who was accused of using an accomplice in the audience to win the top prize. The show
        received a 95% Rotten Tomatoes rating and is noted for its exceptional performances
        and tension-filled narrative.""",
        "category": "Entertainment",
        "source": "expresscouk",
        "pub_date": "2026-01-29"
    },
    {
        "title": "K League Unveils New Super Cup Trophy and Champions' Sleeve Patches",
        "description": """South Korea's K League has revealed the new trophy design for the 2026
        Coupang Play K League Super Cup, marking the competition's return after a 20-year hiatus.
        The match, scheduled for February 21, will feature Jeonbuk Hyundai Motors against
        Daejeon Hana Citizen. Additionally, a new custom sleeve patch system for reigning champions
        has been introduced, incorporating club-specific elements like stadium architecture
        and historical achievements rather than a generic league logo.""",
        "category": "Sports",
        "source": "yahoo_sports",
        "pub_date": "2026-01-29"
    },
    {
        "title": "Kerala Government Approves Multimodal Logistics Parks Near Vizhinjam Port",
        "description": """In a strategic move to boost the industrial ecosystem around the Vizhinjam
        transshipment hub, the Kerala state government has granted in-principle approval for
        logistics parks. Central PSUs like Concor, Indian Oil Corporation, and the Central Warehousing
        Corporation will develop these facilities on over 95 acres of land. These parks are expected
        to act as anchor investments, attracting international shipping and industrial players
        to the region's burgeoning maritime economy.""",
        "category": "Infrastructure",
        "source": "toi",
        "pub_date": "2026-02-08"
    },
    {
        "title": "Tree Walk Volunteers Document Rare Species in Thiruvananthapuram",
        "description": """Environmental volunteers from the 'Tree Walk' group identified and documented
        ten rare trees in the Museum compound of Thiruvananthapuram. The effort aims to promote an
        urban screening policy for indigenous and rare species, such as the Malabar ironwood and
        the Humboldt tree. Participants discussed the importance of scientific pruning and the
        protection of urban greenery against reckless cutting in the name of safety.""",
        "category": "Environment",
        "source": "toi",
        "pub_date": "2026-02-08"
    },
    {
        "title": "Zelensky Commends Workers for Maintaining Infrastructure Under Attack",
        "description": """Ukrainian President Volodymyr Zelensky has issued a formal thank you to
        railway workers and emergency rescuers for their resilience. Despite daily Russian strikes
        targeting railway lines and energy facilities in regions like Kharkiv, Dnipro, and Chernihiv,
        logistics and power remain operational. Zelensky highlighted the names of several emergency
        service employees who have distinguished themselves in maintaining the country's backbone
        during the winter offensive.""",
        "category": "Global Conflict",
        "source": "ukrinform",
        "pub_date": "2026-02-08"
    },
    {
        "title": "Super Bowl LX Predictions: Seahawks vs. Patriots in Prediction Markets",
        "description": """Analysis from the Kalshi prediction market shows the Seattle Seahawks as
        the favorite (68% probability) against the New England Patriots (33%) for Super Bowl LX.
        Unlike traditional sportsbooks, prediction markets allow for peer-to-peer trading on spreads
        and totals. Currently, the point total is hovering around 45.5, indicating market uncertainty
        about whether the game will be a defensive battle or a high-scoring shootout.""",
        "category": "Sports",
        "source": "nj_com",
        "pub_date": "2026-02-08"
    },
    {
        "title": "VAR Controversy in Manchester City's 2-1 Victory Over Liverpool",
        "description": """A bizarre refereeing decision marked Manchester City's win at Anfield.
        Dominik Szoboszlai was sent off in the 103rd minute for a foul on Erling Haaland, despite
        a goal being scored in the same play. Both Pep Guardiola and Haaland expressed confusion
        over the dismissal, calling for 'common sense'. Liverpool manager Arne Slot accepted the
        red card based on the rules but criticized the inconsistency regarding a previous
        non-call on Marc Guehi.""",
        "category": "Sports",
        "source": "yahoo_sports",
        "pub_date": "2026-02-08"
    },
    {
        "title": "Breezy Johnson Wins Gold at Milan Cortina Olympics Amid Vonn's Crash",
        "description": """American skier Breezy Johnson secured the first gold medal for the U.S.
        in the women's downhill at the Milan Cortina Winter Olympics. Her victory was bittersweet
        as it was overshadowed by a frightening crash from teammate Lindsey Vonn. The games also
        marked the return of NHL players to Olympic ice, with teams from the U.S., Canada,
        and Sweden beginning their practice sessions in Milan.""",
        "category": "Sports",
        "source": "sootoday",
        "pub_date": "2026-02-08"
    },
    {
        "title": "Sri Lanka Tops Group B in T20 World Cup After Defeating Ireland",
        "description": """Sri Lanka opened their T20 World Cup 2026 campaign with a 20-run victory
        over Ireland in Colombo. Kamindu Mendis and Kusal Mendis provided a fiery 67-run partnership
        to reach 163. Despite a hamstring injury, Wanindu Hasaranga led the bowling attack with
        3 for 25, supported by Maheesh Theekshana. The win places Sri Lanka at the top of
        the Group B points table with a net run rate of +1.000.""",
        "category": "Sports",
        "source": "india_com",
        "pub_date": "2026-02-08"
    },
    {
        "title": "Experts Dismiss Elon Musk's Theory on Obsolescence of Retirement Saving",
        "description": """Retirement experts have labeled Elon Musk's recent comments 'nonsense'
        after the billionaire suggested that AI and robotics would make personal saving unnecessary.
        Musk envisions a 'universal high income' world of abundance, but researchers from Boston
        College argue that this ignores the realities of health expenses, market volatility,
        y potential shifts in Social Security. They emphasize that for the average worker,
        saving remains the only reliable protection against economic instability.""",
        "category": "Finance",
        "source": "benzinga",
        "pub_date": "2026-02-08"
    },
    {
        "title": "Michigan Dominates Ohio State in Rivalry Win; Aday Mara Shines",
        "description": """The No. 2 Michigan Wolverines maintained their undefeated road record with
        an 82-61 victory over Ohio State. 7'3" center Aday Mara delivered a career-high 24 points,
        demonstrating rare passing vision and perimeter shooting for his size. Michigan's
        overwhelming physical presence in the paint and unselfish ball movement (20 assists)
        proved too much for the Buckeyes' defense, securing a season sweep for the Maize and Blue.""",
        "category": "Sports",
        "source": "si_wolverines",
        "pub_date": "2026-02-08"
    },
    {
        "title": "FBI Files Show No Evidence of Sex Trafficking Ring in Epstein Case",
        "description": """Internal Justice Department records released under the Epstein Files
        Transparency Act reveal that while the FBI found extensive proof of sexual abuse by
        Jeffrey Epstein, they found no evidence of a 'client list' or a sex trafficking ring
        serving powerful men. Investigations into thousands of videos and financial records
        did not implicate high-profile associates in criminal activity. The files clarify why
        federal authorities closed the case without additional charges against influential
        figures mentioned in public allegations.""",
        "category": "Legal",
        "source": "wishtv",
        "pub_date": "2026-02-08"
    }
]

# Step 3: Implement Three Chunking Strategies

In [5]:
def fixed_size_chunks(text, chunk_size=100, overlap=20):
    """Split text into fixed-size chunks with overlap"""
    words = text.split()
    chunks = []

    for i in range(0, len(words), chunk_size - overlap):
        chunk_words = words[i:i + chunk_size]
        if chunk_words:  # Only add non-empty chunks
            chunks.append(' '.join(chunk_words))

    return chunks

def sentence_chunks(text, max_sentences=3):
    """Group sentences into chunks"""
    import re
    sentences = re.split(r'[.!?]+', text)
    sentences = [s.strip() for s in sentences if s.strip()]

    chunks = []
    for i in range(0, len(sentences), max_sentences):
        chunk_sentences = sentences[i:i + max_sentences]
        if chunk_sentences:
            chunks.append('. '.join(chunk_sentences) + '.')

    return chunks

def paragraph_chunks(text):
    """Split by paragraphs or double line breaks"""
    chunks = [chunk.strip() for chunk in text.split('\n\n') if chunk.strip()]
    return chunks if chunks else [text]  # Fallback to full text

# Step 4: Create Collections and Process Data

In [6]:
collection_name = "day1_semantic_search"

if client.collection_exists(collection_name=collection_name):
    client.delete_collection(collection_name=collection_name)

# Create a collection with three named vectors
client.create_collection(
    collection_name=collection_name,
    vectors_config={
        "fixed": models.VectorParams(size=384, distance=models.Distance.COSINE),
        "sentence": models.VectorParams(size=384, distance=models.Distance.COSINE),
        "paragraph": models.VectorParams(size=384, distance=models.Distance.COSINE),
    },
)

# Index fields for filtering (more on this on day 2)
client.create_payload_index(
    collection_name=collection_name,
    field_name="chunk_strategy",
    field_schema=models.PayloadSchemaType.KEYWORD,
)

# Process and upload data
points = []
point_id = 0

for item in my_dataset:
    description = item["description"]

    # Process with each chunking strategy
    strategies = {
        "fixed": fixed_size_chunks(description),
        "sentence": sentence_chunks(description),
        "paragraph": paragraph_chunks(description),
    }

    for strategy_name, chunks in strategies.items():
        for chunk_idx, chunk in enumerate(chunks):
            # Create vectors for this chunk
            vectors = {strategy_name: encoder.encode(chunk).tolist()}

            points.append(
                models.PointStruct(
                    id=point_id,
                    vector=vectors,
                    payload={
                        **item,  # Include all original metadata
                        "chunk": chunk,
                        "chunk_strategy": strategy_name,
                        "chunk_index": chunk_idx,
                    },
                )
            )
            point_id += 1

client.upload_points(collection_name=collection_name, points=points)
print(f"Uploaded {len(points)} chunks across three strategies")

Uploaded 54 chunks across three strategies


# Step 5: Test and Compare

In [7]:
def compare_search_results(query):
    """Compare search results across all chunking strategies"""
    print(f"Query: '{query}'\n")

    for strategy in ["fixed", "sentence", "paragraph"]:
        results = client.query_points(
            collection_name=collection_name,
            query=encoder.encode(query).tolist(),
            using=strategy,
            limit=3,
        )

        print(f"--- {strategy.upper()} CHUNKING ---")
        for i, point in enumerate(results.points, 1):
            print(f"{i}. {point.payload['title']}")
            print(f"   Score: {point.score:.3f}")
            print(f"   Chunk: {point.payload['chunk'][:80]}...")
        print()


# Test with domain-specific queries
test_queries = [
    "country budget",
    "winnnig the cup",
    "robots",
]

for query in test_queries:
    compare_search_results(query)

Query: 'country budget'

--- FIXED CHUNKING ---
1. Thailand Budget Bureau Warns of Debt Ceiling Risks Amid Campaign Pledges
   Score: 0.492
   Chunk: Thailand's Budget Bureau has flagged concerns over political parties' campaign p...
2. Liberia: Senator Crayton Duncan Indicts Legislature for National Poverty
   Score: 0.332
   Chunk: Senator Crayton O. Duncan of Sinoe County has sparked a national debate by attri...
3. Experts Dismiss Elon Musk's Theory on Obsolescence of Retirement Saving
   Score: 0.254
   Chunk: Retirement experts have labeled Elon Musk's recent comments 'nonsense' after the...

--- SENTENCE CHUNKING ---
1. Thailand Budget Bureau Warns of Debt Ceiling Risks Amid Campaign Pledges
   Score: 0.492
   Chunk: Thailand's Budget Bureau has flagged concerns over political parties' campaign 
...
2. Liberia: Senator Crayton Duncan Indicts Legislature for National Poverty
   Score: 0.345
   Chunk: Senator Crayton O. Duncan of Sinoe County has sparked a national debate by 
    

# Step 6: analyzing results

In [8]:
def analyze_chunking_effectiveness():
    """Analyze which chunking strategy works best for your domain"""

    print("CHUNKING STRATEGY ANALYSIS")
    print("=" * 40)

    # Get chunk statistics for each strategy
    for strategy in ["fixed", "sentence", "paragraph"]:
        # Count chunks per strategy
        results = client.scroll(
            collection_name=collection_name,
            scroll_filter=models.Filter(
                must=[
                    models.FieldCondition(
                        key="chunk_strategy", match=models.MatchValue(value=strategy)
                    )
                ]
            ),
            limit=100,
        )

        chunks = results[0]
        chunk_sizes = [len(chunk.payload["chunk"]) for chunk in chunks]

        print(f"\n{strategy.upper()} STRATEGY:")
        print(f"  Total chunks: {len(chunks)}")
        print(f"  Avg chunk size: {sum(chunk_sizes)/len(chunk_sizes):.0f} chars")
        print(f"  Size range: {min(chunk_sizes)}-{max(chunk_sizes)} chars")


analyze_chunking_effectiveness()

CHUNKING STRATEGY ANALYSIS

FIXED STRATEGY:
  Total chunks: 16
  Avg chunk size: 439 chars
  Size range: 20-574 chars

SENTENCE STRATEGY:
  Total chunks: 23
  Avg chunk size: 331 chars
  Size range: 73-585 chars

PARAGRAPH STRATEGY:
  Total chunks: 15
  Avg chunk size: 510 chars
  Size range: 424-628 chars


# Step 7: adding metadata filtering

In [11]:
client.create_payload_index(
    collection_name=collection_name,
    field_name="category",
    field_schema=models.PayloadSchemaType.KEYWORD,
)

results = client.query_points(
    collection_name=collection_name,
    query=encoder.encode("world champion").tolist(),
    using="sentence",
    query_filter=models.Filter(
        must=[
            models.FieldCondition(key="category", match=models.MatchValue(value="Sports"))
        ]
    ),
    limit=3
)

In [14]:
for i, point in enumerate(results.points, 1):
            print(f"{i}. {point.payload['title']}")
            print(f"   Score: {point.score:.3f}")
            print(f"   Chunk: {point.payload['chunk'][:80]}...")

1. K League Unveils New Super Cup Trophy and Champions' Sleeve Patches
   Score: 0.331
   Chunk: South Korea's K League has revealed the new trophy design for the 2026 
        ...
2. Sri Lanka Tops Group B in T20 World Cup After Defeating Ireland
   Score: 0.328
   Chunk: Sri Lanka opened their T20 World Cup 2026 campaign with a 20-run victory 
      ...
3. Breezy Johnson Wins Gold at Milan Cortina Olympics Amid Vonn's Crash
   Score: 0.319
   Chunk: American skier Breezy Johnson secured the first gold medal for the U. S. in the ...
